In [ ]:
import os
import json
import time
import shutil
import rasterio
import urllib.request
import shapely
import numpy as np
import geopandas as gpd
import matplotlib.pylab as plt

from utils import crop_raster, merge_tiles, stitch_tiles, get_tiles

import warnings
warnings.filterwarnings('ignore')

In [ ]:
START_DATE = os.getenv('START_DATE', '2016-05-01')
END_DATE = os.getenv('END_DATE', '2023-06-30')
AOI = os.getenv('AOI', 'POLYGON ((-85.299088 40.339368, -85.332047 40.241477, -85.134979 40.229427, -85.157639 40.34146, -85.299088 40.339368))')
# AOI = os.getenv('AOI', 'MULTIPOLYGON (((-110.49367408906883 39.1005200348037, -110.20673076923077 39.1005200348037, -110.16573886639677 38.41732165423689, -110.43901821862349 38.41732165423689, -110.49367408906883 39.1005200348037)))')

SATELLITE_CACHE_FOLDER = os.getenv('SENTINEL2_CACHE', os.path.join("code", "sentinel_cache", "landcover_dataset"))
OUTPUT_FOLDER = os.getenv('OUTPUT_FOLDER', os.path.join("code", "results", "landcover"))

In [ ]:
polygon = shapely.wkt.loads(AOI)
aoi_filename = f"{time.time()}_aoi.geojson"
gpd.GeoDataFrame(
    gpd.GeoSeries([polygon]),
    columns=["geometry"]).to_file(aoi_filename, driver="GeoJSON")

aoi = gpd.read_file(aoi_filename)

In [ ]:
tile_idx = get_tiles(aoi_filename, "sentinel2grid.geojson")

start_year = int(START_DATE.split("-")[0])
end_year = int(END_DATE.split("-")[0])

date_range = range(2017, 2022)

if end_year in date_range:
    year = end_year
elif start_year in date_range:
    year = start_year
else:
    year = max(list(date_range))

In [ ]:
year

In [ ]:
files = []
landcover_dataset_path = os.path.join(SATELLITE_CACHE_FOLDER, "landcover_dataset")
os.makedirs(landcover_dataset_path, exist_ok=True)

for tile_i in tile_idx.tileID:
    tile_i = tile_i if len(tile_i) < 4 else tile_i[:3]
    tile_url = f"https://lulctimeseries.blob.core.windows.net/lulctimeseriespublic/lc{year}/{tile_i}_{year}0101-{year+1}0101.tif"
    path = os.path.join(landcover_dataset_path, os.path.basename(tile_url))
    if not os.path.exists(path):
        urllib.request.urlretrieve(tile_url, path)
    files.append(path)
    
files

In [ ]:
class_names = {
    1: "Water",
    2: "Trees",
    4: "Flooded vegetation",
    5: "Crops",
    7: "Built Area",
    8: "Bare ground",
    9: "Snow/Ice",
    10: "Clouds",
    11: "Rangeland"
}

In [ ]:
if len(files) > 1:
    try:
        raster_path = merge_tiles(
            files, os.path.join(SATELLITE_CACHE_FOLDER, "aoi_raster.tif"))
    except Exception:
        raster_path = stitch_tiles(
            files, os.path.join(SATELLITE_CACHE_FOLDER, "aoi_raster.tif"))
else:
    raster_path = files[0]

In [ ]:
raster_path = crop_raster(
    raster_path,
    aoi_filename,
    raster_path.replace(".tif", "_crop.tif"),
    additional_meta={
        "START_DATE": f"{year}-01-01",
        "END_DATE": f"{year+1}-01-01",
        "NAME": "Landcover classification"})

with rasterio.open(raster_path) as src:
    img = src.read(1)
    mask = src.read_masks(1)
    profile = src.profile

In [ ]:
NUM_CLASSES = len(class_names)
arr = np.array(range(0, NUM_CLASSES)) / NUM_CLASSES
colors = plt.cm.jet(arr)
colors[0] = (0,0,0,1)

labels = []

for label, name in enumerate(class_names):
    class_area = len(img[img == label]) / 10 ** 4
    # convert list of float values into string representing color
    class_color = ",".join(list(map(lambda x: str(int(x)), colors[label][:-1] * 255)))
    if class_area != 0:
        labels.append({
            "color": class_color, 
            "name": class_names[name],
            "area": class_area
        })
    else:
        labels.append({
            "color": class_color, 
            "name": "Other" if name not in class_names.keys() else class_names[name],
            "area": 0.0
        })
labels = json.dumps(labels)
labels

In [ ]:
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

NUM_CLASSES = 11
nodata = 0
mask = mask.astype(bool)
scaled = img.astype(np.float32) / NUM_CLASSES
scaled = (plt.cm.jet(scaled)[:,:,:-1] * 255)
scaled[mask[:,:,np.newaxis] & (scaled==0)] += 1
scaled = np.clip(scaled, 0, 255).astype(np.uint8)
# Set pixels with invalid pixels to new nodata value
scaled[~mask] = nodata
# Set pixels with background class(0) to new nodata value
scaled[img==0] = nodata

profile.update(
    count=3,
    nodata=nodata,
    compress='lzw'
)
colored_tif = os.path.join(OUTPUT_FOLDER, f"{START_DATE}_{END_DATE}.tif")

with rasterio.open(colored_tif, 'w', **profile) as dst:
    dst.update_tags(start_date=START_DATE, 
                    end_date=END_DATE, 
                    labels=labels,
                    name="Landcover")
    for i in range(scaled.shape[-1]):
        dst.write(scaled[:,:,i], indexes=i+1)

In [ ]:
os.remove(raster_path)
os.remove(aoi_filename)